# Deploy Detection Model

This notebook provides a basic introduction to deploying a trained model as either an ACI or AKS webservice with AML leveraging the azure_utils and tfod_utils packages in this repo.

Before executing the code please ensure you have a completed experiement with a trained model using either the scripts in src or the train model notebooks.

Note that this notebook makes use of additional files in this repo:
- utils - contains mapping functions for the classes that are needed for the deployment image
- conda_env.yml - contains the environment and packages required
- score.py - the base scoring file that is used to created the model_score.py used int he dpeloyment

In [ ]:
import os
import sys

from azure_utils.azure import load_config
from azure_utils.deployment import AMLDeploy


## 1. Define Run Paramters

In [ ]:
# Run params    
ENV_CONFIG_FILE = "dev_config.json"
EXPERIMENT = "pothole"
RUN_ID = "pothole_1629819580_7ce6a2e8"
IMAGE_TYPE = "testpotholeservice"
COMPUTE_TARGET_NAME = "testdeployment"
MODEL_NAME = "testpotholeservice"
WEBSERVICE_NAME = MODEL_NAME.lower().replace("_", '')



## 2. Initialise Deployment Class

In [ ]:
deployment = AMLDeploy(RUN_ID,
                       EXPERIMENT,
                       WEBSERVICE_NAME,
                       MODEL_NAME,
                       IMAGE_TYPE,
                       config_file=ENV_CONFIG_FILE)

## 3. Register Model from Experiment

In [ ]:
model = deployment.register_run_model()

## 4. Set Scoring Script

The base score file is avliable in the src dir, variation can be created as needed. At deployment model name will be updated to create the final deploy script.

We also set the src dir to the deployment src folder so that at deployment we can access the utils

In [ ]:
src_dir = os.path.join('..', 'src', 'deployment')
score_file = os.path.join(src_dir, 'score_tf2.py')
env_file = './conda_env_tf2.yml'

## 5. Create Inference Config

In [ ]:
inference_config = deployment.create_inference_config(score_file, src_dir, env_file)

## 6. Check is a webservice exists with same name

Checks if there is a webservice with the same name. If it returns true you can either skip the next two cells and update that service or change the service name to deploy a new webservice.

In [ ]:
deployment.webservice_exists(deployment.webservice_name)

## 6. Deploy ACI

Deploy the model to an ACI endpoint, this will be targeting a CPU instance and not GPU and is used just for testing purposes.

In [ ]:
target, config = deployment.create_aci()

deployment.deploy_new_webservice(model,
                                 inference_config,
                                 config,
                                 target)

## 7. Deploy AKS

In [ ]:
deployment.webservice_name = deployment.webservice_name + "-aks"

target, config = deployment.create_aks(COMPUTE_TARGET_NAME, exists=False)

deployment.deploy_new_webservice(model,
                                 inference_config,
                                 config,
                                 target)

## 8. Update Existing Webservice

In [ ]:
deployment.update_existing_webservice(model, inference_config)